# Introduction
In the last decades, the sport of sailing has experienced an increasing impact
of new technologies, and notably of scientific computing. Among all computational problems relevant for sailing, we are interested here in route planning
and race strategy, i.e., the optimization of the yacht route.
	
This project focuses on developing algorithms that address the issue of optimizing a sailboat’s trajectory when a starting point and destination are given alongside static wind conditions. The underlying physics that govern the optimal path of a sailboat for a given set of conditions are highly coupled and dynamic, rendering the course very unintuitive to determine. Algorithms that are able to produce the path plan that takes the minimum amount of time to complete the course can be very helpful.
	
Algorithms developed in this project use the idea of calculating the Velocity Made Good (VMG) as a parameter relating the state of the sailboat at any given time to the time it would take to complete a given course. 
	
Before going into details about how the models work, some basic sailing theory is introduced. It is not possible for boats to sail directly into the wind, requiring the course of the boat to alternate between headings. This process is called "tacking" and is used commonly by sailors to make their way to a mark that is upwind. On a tack, the sailor will generally point the sailboat as close to the wind as possible while still keeping the winds blowing through the sails in a manner that provides aerodynamic lift to propel the boat.
	
Then the boat is turned away from the wind in slight increments in order to generate more forward lift on the sails, allowing it to move with greater speed, but less directly toward the destination. The range of heading that does not produce any significant lift is called the no-go zone.

<img src="https://github.com/Lucmeister55/STMO-assignment/blob/main/images/sailing_intro.png?raw=true"  width="600" height="600">

## VMG
The VMG can be calculated by using the following expression where Vtrue is the velocity of the sailboat with respect to stationary ground and 𝜃 is the angle between current heading and the direction to destination. 

$$VMG=Vtrue∗ cos\theta$$

However, the relationship between the true velocity of the sailboat and the true wind velocity depends on what assumptions are made. Some instances take into consideration the fact that a sailboat’s Speed Over Ground increases or decreases relative to the wind direction. In theory, a sailboat’s speed increases while sailing from upwind to a downwind direction. Other parameters to factor in include the sail boat’s specifics that depend on the make and design of the boat. These are the 'Velocity Increase Constant', ‘No – Go Zone’ and ‘Degree Interval’, which are normally provided by the manufacturer. The physical meaning of this constant expresses 'the sail boat increases speed by 10% for every X degrees from the wind'. Considering these factors, a True VMG can be calculated using the equations:

For Upwind: $VMG=\frac{V_w}{cos\theta_s}*(1+\beta)^{\frac{|\theta_0-\theta_s|}{i}}*\theta_\gamma$

For Downwind: $VMG=\frac{V_w}{cos\theta_s}*(1+\beta)^{\frac{|180°-\theta_0-\theta_s|}{i}}*\theta_\gamma$

VMG = Velocity Made Good towards destination\
$V_w$ = Velocity of wind\
$\theta_s$ = Angle between heading and direction\
$\theta_0$ = No-go zone\
$\theta_\gamma$ = Angle between wind direction and heading\
$\beta$ = Velocity increase constant\
i = Degree Interval

In the expressions shown above the no-go zone ($\theta_0$), Degree Interval (i) and Velocity Increase Constant ($\beta$) are usually provided by the manufacturer. As these are specific to the boat’s design, commonly quoted values are used throughout this project.

## Assumptions
In order to simplify the optimization problem, the following assumptions are made and incorporated into the models:
- True velocity of boat at any point of time is a function of the velocity of wind and heading relative to wind direction only. This implies that at any point these are the only two variables required to calculate how fast the boat will be moving with respect to ground (true velocity) in the direction it is moving.
- Effects of momentum are not considered. As the expressions shown earlier allow for the calculation of velocity instantaneously, the time-dynamic effects are not modelled. This means that acceleration and deceleration are not taken into account and if the boat moves from a current position to the next position, the momentum is not carried or lost but the position’s assigned momentum is taken up.
- Effects of drag are not considered. The effects of drag considered are only those specified by the velocity increase constant and the no-go zone, both provided by the manufacturer. The effects of drag due to the wind and the water due to the form of the boat are not taken into consideration. In real life however, this phenomenon will have significant effects limiting the maximum velocity of the sailboat.
- Effects of manoeuvres on the momentum of the sailboat are not considered. For example, tacking causes the sailboat to lose momentum due to the associated drag and loss of lift during the procedure. The loss of momentum is a transient process and is not modelled for this project.

## Decision Variables
The objective of the algorithms is to produce paths that take the least amount of time to complete a given course. In order to describe the path taken by the sailboats, the variables required are the x and y coordinates of the sailboat at a given time. Instead of determining the coordinates for each time step, 
they are determined in consecutive steps-this means the coordinates $X^{k+1}$ are determined relative to $X^k$ in distance and direction as opposed to determining $X^{k+1}$ as a function of time step $t^{k+1}$.

The problem of producing a locus of X, Y coordinates that are successive in nature can be transformed into producing a direction vector and a distance vector from each of the points to the next point. An example of this is- if $X^k$ is known, in order to determine the next position $X^{k+1}$ a direction vector and a step size are sufficient and required with relation to $X^k$. Hence, the modelling problem is reduced to finding the optimal sequence of direction vectors (heading) if the step size is set to be constant. 

In other standard optimization problems tackled during the course, the design space did not include a time dimension. However, this problem indeed has a time dimension and the variable (heading) will have to be determined as the time marches forward (in distance steps and not time steps).

## Objective
The objective of the algorithm is to find the path of least resistance, or the path that takes the least amount of time to complete a given course. The time taken is calculated in a discretized manner: The velocity of the sailboat is found for each step and is used to divide the step size.

$\textrm{time step k} = \frac{\textrm{step size}}{\textrm{velocity in direction of step}}$

The time steps are added for the duration of the course in order to register the total time taken.

$\textrm{time elapsed} = \sum{\textrm{time step k}}$

The objective is to optimize the path to achieve the lowest time elapsed value for any given course. In order to study the behaviour of the model and the algorithms, 7 cases are used to investigate. In addition to reducing this parameter, one of the models is developed to first prioritize ease of navigation and then minimize the time elapsed parameter. The difference in performances of both models is investigated to understand the trade-off. The ease of navigation cannot be quantified; hence 
both the models are inherently different.

## Constraints
This path optimization problem only has one constraint – at no instance should the boat bear a heading into the no-go zone. In real life however, during a tack, the boat indeed faces the no-go zone briefly before regaining lift in the sails. It is its momentum that allows the boat to steer away from the no-go zone and into a heading that permits forward motion. In the models presented in the report, it is 
important to remember that the boat’s momentum terms are not incorporated. This means that if at any point the algorithm produces a path that involves a heading into the no-go zone, it loses its velocity and cannot make further steps. The algorithms get past this flaw by limiting the heading from entering the no-go zone at all times.

When travelling upwind,

$$\theta_w > \theta_0$$

$\theta_0$ = no-go zone\
$\theta_w$ = heading relative to wind direction

## Model
The Single Tack Method (STM) is developed so that the path produced by the model is extremely easy to navigate. This is achieved by implicitly allowing the algorithm to produce a path between two points that incorporates a maximum of one tack (i.e. two leg journey). The algorithm is then required to produce the tack length and angle that results in the minimum time elapsed value. It is important to note that the manner in which ease of navigation is incorporated into this model is by limiting the number of tacks. By increasing the number of tacks, the solution process can get very complex. 

To explain this, a scenario is discussed – If arbitrary location A is considered the starting point and a destination is set at B, to produce a path that can only have a single tack, the problem is reduced to finding a point Ts (tack point) such that when the boat travels from A to Ts, Ts to B, it takes less amount 
of time than if the boat sailed from A to T, T to B. Here T is an arbitrary tack location. If two tacks were allowed, this would mean that the algorithm would have to find the best combination of two points that result in the smallest time elapsed value. Due to the highly coupled nature of the problem, navigating through two dimensions to find the best combination would be very difficult, and hence is out of the scope of this project.

# Implementation

In [2]:
import math
import numpy as np
import ipywidgets as widgets
from ipywidgets import HBox, VBox
import matplotlib.pyplot as plt
from IPython.display import display

## Objective Function

In [34]:
def my_acos(x):
    return math.acos(x) if math.isclose(x, 1) else (math.pi if math.isclose(x, -1) else math.acos(x))

def pathtime(x_tar_c, x0, xt, vel_wind, wind_dir, step_inc):
    theta_nogo = math.radians(40) # deadzone
    velcons = 3 # velocity increase constant
    deg_int = 5 # constant provided by manufacture
    
    # tack vector 1
    tack_vect_1 = [xt[i] - x0[i] for i in range(len(x0))]
    # tack vector 2
    tack_vect_2 = [x_tar_c[i] - xt[i] for i in range(len(xt))]
    
    # divide it into increments of .1 metres
    steps_1 = math.sqrt(sum([(xt[i] - x0[i])**2 for i in range(len(x0))])) / step_inc
    # set initial step
    step_c_1 = 0
    heading = tack_vect_1
    xc = x0
    time_elapsed_1 = 0
    
    while step_c_1 < steps_1:
        # vector to target from current x
        target_vect = [x_tar_c[i] - xc[i] for i in range(len(xc))]
        
        # calculate angle between heading and target vector
        theta_rd = my_acos(sum([heading[i] * target_vect[i] for i in range(len(heading))]) / (math.sqrt(sum([heading[i]**2 for i in range(len(heading))])) * math.sqrt(sum([target_vect[i]**2 for i in range(len(target_vect))]))))
        
        # calculate angle between heading and wind direction
        theta_rw = my_acos(sum([heading[i] * wind_dir[i] for i in range(len(heading))]) / (math.sqrt(sum([heading[i]**2 for i in range(len(heading))])) * math.sqrt(sum([wind_dir[i]**2 for i in range(len(wind_dir))]))))
        
        if sum([wind_dir[i] * heading[i] for i in range(len(wind_dir))]) / (math.sqrt(sum([wind_dir[i]**2 for i in range(len(wind_dir))])) * math.sqrt(sum([heading[i]**2 for i in range(len(heading))]))) < 0:
            # calculate velocity made good
            u = math.cos(theta_rd) * (vel_wind / math.cos(theta_nogo)) * (1 + 0.01 * velcons) ** ((abs(theta_rw - theta_nogo) * 180) / (math.pi * deg_int))
            # check if within deadzone
            v = u if math.degrees(my_acos(sum([heading[i] * wind_dir[i] for i in range(len(heading))]) / (math.sqrt(sum([heading[i]**2 for i in range(len(heading))])) * math.sqrt(sum([wind_dir[i]**2 for i in range(len(wind_dir))]))))) < 140 else 0
        elif sum([wind_dir[i] * heading[i] for i in range(len(wind_dir))]) / (math.sqrt(sum([wind_dir[i]**2 for i in range(len(wind_dir))])) * math.sqrt(sum([heading[i]**2 for i in range(len(heading))]))) > 0:
            # calculate velocity made good if downwind
            v = math.cos(theta_rd) * (vel_wind / math.cos(theta_nogo)) * (1 + 0.01 * velcons) ** ((abs(math.pi - theta_rw - theta_nogo) * 180) / (math.pi * deg_int))
        
        vel_tack = v / math.cos(theta_rd)
        
        if vel_tack > 10e-2:
            # time elapsed in increment
            time_increment_1 = 0.1 / vel_tack 
            time_elapsed_1 += time_increment_1
            xc = [0.1 * heading[i] / math.sqrt(sum([heading[j]**2 for j in range(len(heading))])) + xc[i] for i in range(len(xc))]
            step_c_1 += 1
        elif vel_tack < 10e-2:
            time_elapsed_1 = 10e6
            break
    
    t_1 = time_elapsed_1
    
    # divide it into increments of .1 metres
    steps_2 = math.sqrt(sum([(x_tar_c[i] - xt[i])**2 for i in range(len(xt))])) / step_inc
    # set initial step
    step_c_2 = 0
    heading = tack_vect_2
    xc = xt
    time_elapsed_2 = 0
    
    while step_c_2 < steps_2:
        target_vect = [x_tar_c[i] - xc[i] for i in range(len(xc))]
        theta_rd = my_acos(sum([heading[i] * target_vect[i] for i in range(len(heading))]) / (math.sqrt(sum([heading[i]**2 for i in range(len(heading))])) * math.sqrt(sum([target_vect[i]**2 for i in range(len(target_vect))]))))
        theta_rw = my_acos(sum([heading[i] * wind_dir[i] for i in range(len(heading))]) / (math.sqrt(sum([heading[i]**2 for i in range(len(heading))])) * math.sqrt(sum([wind_dir[i]**2 for i in range(len(wind_dir))]))))
        if sum([wind_dir[i] * heading[i] for i in range(len(wind_dir))]) / (math.sqrt(sum([wind_dir[i]**2 for i in range(len(wind_dir))])) * math.sqrt(sum([heading[i]**2 for i in range(len(heading))]))) < 0:
            u = math.cos(theta_rd) * (vel_wind / math.cos(theta_nogo)) * (1 + 0.01 * velcons) ** ((abs(theta_rw - theta_nogo) * 180) / (math.pi * deg_int))
            v = u if math.degrees(my_acos(sum([heading[i] * wind_dir[i] for i in range(len(heading))]) / (math.sqrt(sum([heading[i]**2 for i in range(len(heading))])) * math.sqrt(sum([wind_dir[i]**2 for i in range(len(wind_dir))]))))) < 140 else 0
        elif sum([wind_dir[i] * heading[i] for i in range(len(wind_dir))]) / (math.sqrt(sum([wind_dir[i]**2 for i in range(len(wind_dir))])) * math.sqrt(sum([heading[i]**2 for i in range(len(heading))]))) > 0:
            v = math.cos(theta_rd) * (vel_wind / math.cos(theta_nogo)) * (1 + 0.01 * velcons) ** ((abs(math.pi - theta_rw - theta_nogo) * 180) / (math.pi * deg_int))
        vel_tack = v / math.cos(theta_rd)
        if vel_tack > 10e-2:
            time_increment_2 = 0.1 / vel_tack
            time_elapsed_2 += time_increment_2
            xc = [0.1 * heading[i] / math.sqrt(sum([heading[j]**2 for j in range(len(heading))])) + xc[i] for i in range(len(xc))]
            step_c_2 += 1
        elif vel_tack < 10e-2:
            time_elapsed_1 = 10e6
            break
    
    t_2 = time_elapsed_2
    t = t_1 + t_2
    return t

## Step Size

In [ ]:
def backtracking_line_search(f, x0, Dx, grad_f, c=0.1,rho=0.7):
    alpha=1
    while (f(x0 + alpha*Dx) > f(x0) +  alpha*c*np.sum(grad_f(x0) *Dx)):
        alpha *=rho
    return alpha

## Tackpoint Estimation

### Line Search (Brute Force)
The starting point and the current destination is specified in this routine. The programme then calculates an initial feasible point for the tack location. The model employs a pattern search algorithm incorporating an accelerating/decelerating step size. The pattern search algorithm requires to start in the feasible region because starting in the no-go zone results in an infinite path time value.

Once, the pattern search algorithm is initialised, the path time is calculated in each of the probe directions (x+,y+,x-,y-) using the function handle pathtime. This function handle accepts the starting point of the sailboat, the tack point and the destination point and integrates along the two legs of the journey to find the time elapsed (or the path time). This is the objective function value that needs to be minimised. 

With the objective function values from all four probes, the pattern search algorithm chooses the directions in x and y that favour the minimizing of the path time and determines the new tack point. Depending on the search directions (in X and Y) recorded, the pattern search algorithm updates the acceleration terms to reduce the number of pattern moves required to achieve convergence. Once convergence is reached, the optimal tack point (xt) and the optimal time elapsed values are returned.

In [31]:
def tackpoint_LS(x0, x_tar_c, maxiter, vel_wind, wind_dir, step_inc):
    # calculating initial xt
    print(x_tar_c)
    print(x0)
    vect_tar = (x_tar_c - x0)
    mid = 0.5 * vect_tar + x0
    deviation = ((x_tar_c[1] - x0[1]) ** 2 / (x_tar_c[0] - x0[0])) / (math.atan(math.radians(15)))
    xt0 = [-deviation, 0] + mid

    xt = xt0
    iter = 0

    # Introducing probes for pattern search
    plength_x = [10, 0]  # probe length in x
    plength_y = [0, 10]  # probe length in y
    ax = 0.1  # initial stepsize
    ay = 0.1  # initial stepsize
    adpxt = 1  # initialize acceleration determining parameter in x
    adpyt = 1  # initialize acceleration determining parameter in y

    while iter < maxiter:
        # calculate path times at probes
        pathtime_xtc = pathtime(x_tar_c, x0, xt, vel_wind, wind_dir, step_inc)
        pathtime_xtc_xplus = pathtime(x_tar_c, x0, xt + plength_x, vel_wind, wind_dir, step_inc)
        pathtime_xtc_xminus = pathtime(x_tar_c, x0, xt - plength_x, vel_wind, wind_dir, step_inc)
        pathtime_xtc_yplus = pathtime(x_tar_c, x0, xt + plength_y, vel_wind, wind_dir, step_inc)
        pathtime_xtc_yminus = pathtime(x_tar_c, x0, xt - plength_y, vel_wind, wind_dir, step_inc)

        # probing in x and updating xt
        # find minimum path time
        if min(pathtime_xtc, pathtime_xtc_xplus, pathtime_xtc_xminus) == pathtime_xtc:
            xt = xt
            sx = 0  # set search direction
            adpx = 0  # set acceleration term
        elif min(pathtime_xtc, pathtime_xtc_xplus, pathtime_xtc_xminus) == pathtime_xtc_xplus:
            sx = 1  # set search direction
            adpx = 1  # set acceleration term
        elif min(pathtime_xtc, pathtime_xtc_xplus, pathtime_xtc_xminus) == pathtime_xtc_xminus:
            sx = -1  # set search direction
            adpx = -1  # set acceleration term

        # probing in y and updating xt
        if min(pathtime_xtc, pathtime_xtc_yplus, pathtime_xtc_yminus) == pathtime_xtc:
            sy = 0  # set search direction
            adpy = 0  # set acceleration term
        elif min(pathtime_xtc, pathtime_xtc_yplus, pathtime_xtc_yminus) == pathtime_xtc_yplus:
            sy = 1  # set search direction
            adpy = 1  # set acceleration term
        elif min(pathtime_xtc, pathtime_xtc_yplus, pathtime_xtc_yminus) == pathtime_xtc_yminus:
            sy = -1  # set search direction
            adpy = -1  # set acceleration term

        iter += 1
        ax = ax * 1.15 ** (adpx * adpxt)  # update acceleration in x
        ay = ay * 1.15 ** (adpy * adpyt)  # update acceleration in y
        xt_p = xt  # update xt previous
        xt = xt + [ax * sx, ay * sy]  # update tack position
        adpxt = adpx  # store acceleration terms
        adpyt = adpy
        minpathtime = pathtime(x_tar_c, x0, xt, vel_wind, wind_dir, step_inc)  # calculate corresponding minpathtime
        minpathtime_p = pathtime(x_tar_c, x0, xt_p, vel_wind, wind_dir, step_inc)

        # prevent entering into infeasible region
        if minpathtime_p < minpathtime:
            xt = xt_p
            minpathtime = minpathtime_p
            ax = 0.1  # reset acceleration
            ay = 0.1  # reset acceleration

        print(iter)
        print(xt)
        print(minpathtime)

    return xt

### Gradient Descent

In [ ]:
import math

def tackpoint_GD(x0, x_tar_c, maxiter, vel_wind, wind_dir, step_inc):
    # Calculating initial xt ---------------------------------------------------------
    vect_tar = (x_tar_c - x0) # Generate vector to active target
    mid = 0.5 * vect_tar + x0 # Place point in the middle
    deviation = ((x_tar_c[1] - x0[1]) ** 2 / (x_tar_c[0] - x0[0])) / (math.atan(math.radians(15))) # Define deviation of point from vect_tar
    xt0 = [-deviation, 0] + mid # Initial tack point
    
    xt = xt0
    
    iter = 0
    
    alpha = 1
    rho = 0.7
    c = 0.1
    
    while iter < maxiter: # Set maximum number of iterations
        # Define loss function
        def loss(xt):
            return pathtime(x_tar_c, x0, xt, vel_wind, wind_dir, step_inc)
        
        # Compute gradients
        def gradient(xt):
            return [Flux.gradient(loss, xt)[0]]
        
        grad_xt = gradient(xt)
        
        t = backtracking_line_search(loss, xt, grad_xt, c, rho)
        
        xt -= alpha * grad_xt
        
        iter += 1
        
        print(iter)
        print(xt)
    
    return xt

### Newton's Method

## Calculate Path
Function to calculate the path produced by any algorithm.

In [32]:
def track(x0, x_tar, target_tol):
    xp = x0  # current position

    # Initialize target tracking
    num_targets = x_tar.shape[0]  # get number of targets
    target_tracker = np.zeros(num_targets, dtype=np.int64)  # create file indicating no targets have been achieved

    itt = 1  # target tracker iteration
    while itt < num_targets + 1:
        target_tracker[itt - 1] = 0
        itt += 1

    x_tar_c = [x_tar[0, 0], x_tar[0, 1]]  # set initial target
    tar_act = 1  # active target is 1

    # Target tracking loop
    x = []
    y = []
    while np.linalg.norm(x_tar_c - xp) > target_tol / 10:  # if target is not achieved
        step_direction = (x_tar_c - xp) / np.linalg.norm(x_tar_c - xp)
        stepsize = 1
        xp_p = xp  # store previous location
        xp = step_direction * stepsize + xp
        x.append(xp[0])
        y.append(xp[1])

        if np.linalg.norm(x_tar_c - xp) < target_tol:
            if tar_act < x_tar.shape[0]:
                target_tracker[tar_act - 1] = 1
                x_tar_c = [x_tar[tar_act, 0], x_tar[tar_act, 1]]
                tar_act += 1

        print(np.linalg.norm(x_tar_c - xp))

    return x, y

# Simulation

## Parameter Options

In [29]:
x0_x_slider = widgets.FloatSlider(
    value=20,
    min=0,
    max=50,
    step=1,
    description='X:',
    readout_format='.1f',
)

x0_y_slider = widgets.FloatSlider(
    value=20,
    min=0,
    max=50,
    step=1,
    description='Y:',
    readout_format='.1f',
)

x_tar_x_slider = widgets.FloatSlider(
    value=0,
    min=0,
    max=50,
    step=1,
    description='X:',
    readout_format='.1f',
)

x_tar_y_slider = widgets.FloatSlider(
    value=50,
    min=0,
    max=50,
    step=1,
    description='Y:',
    readout_format='.1f',
)

vel_wind_slider = widgets.FloatSlider(
    value=10,
    min=0,
    max=10,
    step=1,
    description='Wind Velocity:',
    readout_format='.1f',
)

wind_dir_buttons = widgets.ToggleButtons(
    options=["N", "NE", "E", "SE", "S", "SW", "W", "NW"],
    description='Wind Direction (Origin):',
)

maxiter_slider = widgets.FloatSlider(
    value=100,
    min=1,
    max=1000,
    step=1,
    description='Maximum Iterations:',
    readout_format='.1f',
)

step_inc_slider = widgets.FloatSlider(
    value=0.1,
    min=0.05,
    max=1,
    step=0.05,
    description='Step size:',
    readout_format='.1f',
)

button = widgets.Button(
    description='Plot',
)
button

@button.on_click
def plot_on_click(b):
    plot_LS()

tab1 = VBox(children=[x0_x_slider, x0_y_slider])
tab2 = VBox(children=[x_tar_x_slider, x_tar_y_slider])
tab3 = VBox(children=[vel_wind_slider, wind_dir_buttons])
tab4 = VBox(children=[maxiter_slider, step_inc_slider])

tab = widgets.Tab(children=[tab1, tab2, tab3, tab4])
tab.set_title(0, 'Starting Point')
tab.set_title(1, 'Destination Point')
tab.set_title(2, 'Wind')
tab.set_title(3, 'Algorithm')
VBox(children=[tab, button])

[ 0. 50.]
[20. 20.]


NameError: name 'pathtime' is not defined

[ 0. 50.]
[20. 20.]


ValueError: math domain error

In [30]:
def plot_LS(b=None):
    x0_x = x0_x_slider.value
    x0_y = x0_y_slider.value
       
    x_tar_x = x_tar_x_slider.value
    x_tar_y = x_tar_y_slider.value

    vel_wind = vel_wind_slider.value
    wind_dir_temp = wind_dir_buttons.value

    maxiter = maxiter_slider.value
    step_inc = step_inc_slider.value

    if wind_dir_temp == "N":
        wind_dir = [0, -1]
    elif wind_dir_temp == "NE":
        wind_dir = [-1, -1]
    elif wind_dir_temp == "E":
        wind_dir = [-1, 0]
    elif wind_dir_temp == "SE":
        wind_dir = [-1, 1]
    elif wind_dir_temp == "S":
        wind_dir = [0, 1]
    elif wind_dir_temp == "SW":
        wind_dir = [1, 1]
    elif wind_dir_temp == "W":
        wind_dir = [1, 0]
    else:
        wind_dir = [1, -1]

    # Routine to determine the best tack point between two marks and calculate time
    x_tar_c = np.array([x_tar_x, x_tar_y])  # active destination
    x0 = np.array([x0_x, x0_y])  # starting point
        
    xt = tackpoint_LS(x0, x_tar_c, maxiter, vel_wind, wind_dir, step_inc)
    # calculate single tack path
    target_tol = 4 # radius in metres within target to qualify
    x_tar = np.array([[xt[0], xt[1]], [x_tar_c[0], x_tar_c[1]]])

    x, y = track(x0, x_tar, target_tol)

    plt.scatter(x, y)
    # Plot the wind vector
    xw = [x_tar[1, 0], x_tar[1, 0] + wind_dir[0] * 10]
    yw = [x_tar[1, 1], x_tar[1, 1] + wind_dir[1] * 10]
    plt.plot(xw, yw, '->', markersize=5)
    plt.show()